In [2]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import invwishart

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

np.random.seed(0)

In [3]:
true = np.load('true.npy')
perturbed = np.load('perturbed.npy')

In [4]:
data = true

In [5]:
k = 4
d = 2

In [6]:
def run_one_gibbs(data, cluster_allocs, uniq_vals, weights):
    """
    Runs one iteration of the gibbs sampler
    Parameters
    ----------
    data: a numpy array of floats, contains the observations
    cluster_allocs: a numpy array of integers, contains the c_i's(same length of data)
    uniq_vals: a numpy array of size [K, 2], contains (mean, variance) for every component
    weights: a numpy array of size [K], contains the weights of the components
    """
    
    n_clus = len(weights)
    
    for h in range(n_clus):
        clusdata = data[cluster_allocs == h]
        if len(clusdata) == 0:
            uniq_vals[h, :] = sample_uniq_vals_prior()
        else:
            uniq_vals[h, :] = sample_uniq_vals_fullcond(clusdata)
    
    cluster_allocs = update_cluster_allocs(data, weights, uniq_vals)
    
    weights = update_weights(cluster_allocs, n_clus)
    
    return cluster_allocs, uniq_vals, weights



def run_mcmc(data, k, niter=7500, nburn=2500, thin=5):
    cluster_allocs = tfd.Categorical(probs=np.ones(k) / k).sample(len(data))
    weights = np.ones(k) / k
    
    ### CAPIRE LA INVERSE WISHART
    uniq_vals = np.dstack([
        tfd.MultivariateNormalFullCovariance(np.mean(np.array(data)), np.diag(20*np.ones(d))).sample(k),
        invwishart.rvs(df = d+1, scale = np.diag(2*np.ones(d)),size=k)])
    
    allocs_out = []
    uniq_vals_out = []
    weights_out = []
    
    for i in range(niter):
        cluster_allocs, uniq_vals, weights = run_one_gibbs(
            data, cluster_allocs, uniq_vals, weights)
        
        if i > nburn and i % thin == 0:
            allocs_out.append(cluster_allocs)
            uniq_vals_out.append(uniq_vals)
            weights_out.append(weights)
            
        if i % 10 == 0:
            print("\rIter {0} / {1}".format(i+1, niter), flush=True, end=" ")
            
    return allocs_out, uniq_vals_out, weights_out

In [7]:
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions


def update_cluster_allocs(data, weights, uniq_vals):
    
    logprobs = tfd.Normal(
        uniq_vals[:, 0], np.sqrt(uniq_vals[:, 1])).log_prob(data[:, np.newaxis])
    logprobs += np.log(weights)
    probs =  np.exp(logprobs)/np.sum(np.exp(logprobs))
    return tfd.Categorical(probs=probs).sample()


def update_weights(cluster_allocs, n_clus, k):

    n_by_clus = np.array([np.sum(cluster_allocs == h) for h in range(n_clus)])
    post_params = np.ones(k) / k + n_by_clus
    return tfd.Dirichlet(post_params.astype(float)).sample()


def sample_uniq_vals_prior():
    
    var = invwishart.rvs(df = d+1, scale = np.diag(2*np.ones(d)),size=1)
    mu = tfd.MultivariateNormalFullCovariance(np.mean(np.array(data)), np.diag(20*np.ones(d))).sample()
    return np.array([mu, var])
    

def sample_uniq_vals_fullcond(clusdata):
    
    n=len(clusdata)
    
    var = invwishart.rvs(df = d+1+n, scale = np.diag(2*np.ones(d))+n*np.var(clusdata)+(n/1+n)*np.dot(np.mean(clusdata)-np.mean(data),np.mean(clusdata)-np.mean(data))
                         ,size=1)
    mu = tfd.MultivariateNormalFullCovariance((np.mean(np.array(data))+n*np.mean(clusdata))/(1+n), np.diag(20*np.ones(d))/(1+n)).sample()
    return np.array([mu, var], dtype=object)

In [8]:
allocs_out, uniq_vals_out, weights_out = run_mcmc(data, k)

ValueError: could not broadcast input array from shape (2,2) into shape (2,)

In [ ]:
plt.hist(data, density=True, alpha=0.1)
for h in range(k):
    currd = data[allocs_out[-1] == h]
    plt.scatter(currd, np.zeros_like(currd) + 0.001 * (h+1))